In [1]:
import os
import pandas as pd
from datetime import datetime

In [7]:
# Define Directories
working_dir = r'C:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Inventory\WH Inventory Reports\Working_files'
output_dir = r'C:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Inventory\WH Inventory Reports\Reports'

# Get current month and year
current_year = datetime.now().year
current_month = datetime.now().month

# Find the current month's files 
files = [f for f in os.listdir(working_dir) if f.endswith(".xlsx")]
print(files)

# Read and combine data
all_data = []
for file in files:
    file_path = os.path.join(working_dir,file)
    try:
        df = pd.read_excel(file_path)
        df['Source File'] = file
        all_data.append(df[['Source File','Plant', 'Storage Location', 'Material', 'Description', 'Batch', 'Qty_SAP', 'Qty_WH', 'Delta', 'QC Hold Qty PSS', 'Hold Reason ES3',
                            'Manufacture Date', 'Critical Shelf Life Date', 'Expiration date', 'Number of Summers', 'Unit Value', 'Pallet number']])
    except Exception as e:
        print(f"Failed to process {file}: {e}")

['SAP_vs_Warehouse_all_data_20250204_1029.xlsx', 'SAP_vs_Warehouse_all_data_20250210_0931.xlsx', 'SAP_vs_Warehouse_all_data_20250211_1006.xlsx', 'SAP_vs_Warehouse_all_data_20250218_1232.xlsx', 'SAP_vs_Warehouse_all_data_20250221_1130.xlsx', 'SAP_vs_Warehouse_all_data_20250224_1304.xlsx', 'SAP_vs_Warehouse_all_data_20250226_1029.xlsx']


In [3]:
# Combine into single DataFrame
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)

    # Group by key fields and find constant deltas
    summary = (combined_df.groupby(['Plant', 'Storage Location', 'Material', 'Batch'])
                           .agg({'Delta': ['min', 'max', 'count']})
                           .reset_index())
    summary.columns = ['Plant', 'Storage Location', 'Material', 'Batch', 'Min Delta', 'Max Delta', 'Occurances']

    # Filter for consistent deltas (min = max)
    consistent_deltas = summary[summary['Min Delta'] == summary['Max Delta']]

    # Ouput to Excel
    output_file = os.path.join(output_dir, f"constant_delta_report_{current_year}-{current_month:02d}.xlsx")
    consistent_deltas.to_excel(output_file, index=False)

    print(f"Report generated: {output_file}")
else:
    print("No data to process")


Report generated: C:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Inventory\WH Inventory Reports\Reports\constant_delta_report_2025-02.xlsx


In [4]:
# Not including zero values
consistent_deltas_plus = summary[(summary['Min Delta'] == summary['Max Delta']) & (summary['Min Delta'] != 0)]
consistent_deltas_plus  

,Plant,Storage Location,Material,Batch,Min Delta,Max Delta,Occurances
8,BP01,SL02,GN21401,12424235,-175,-175,1
9,BP01,SL02,GN21401,12424239,-85,-85,1
10,BP01,SL02,GN21401,12424261,-13,-13,1
16,BP01,SL02,GN60GF1,12324195,-68,-68,1
34,BP01,SL02,VIP1034,124088,33,33,1
...,...,...,...,...,...,...,...
1299,BP07,SL02,VSS0041,124298,-70,-70,1
1331,BP07,SL02,VSS0091,124155,-70,-70,1
1363,BP07,SL02,VSS4001,221327,-60,-60,2
1384,BP07,SL06,GN21401,12324180,238,238,2
